In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Any results you write to the current directory are saved as output.

In [2]:
#loading train and test data-set

train = pd.read_csv("D:\data analysis\COMPETITIONS 2020\BIG DATA CHALLENGE\TrainingData.csv",header=0)
test = pd.read_csv("D:\data analysis\COMPETITIONS 2020\BIG DATA CHALLENGE\TestX.csv",header=0)
print(train.shape,test.shape)

(80000, 50) (3000, 49)


C:\Users\SARANSH\Anaconda3\envs\saransh\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#replacing all missing and faulty values by NaN

train = train.replace('missing', np.nan)
train = train.replace('N/A', np.nan)
train = train.replace('na', np.NaN)

test = test.replace('missing', np.nan)
test = test.replace('N/A', np.nan)
test = test.replace('na', np.NaN)

In [4]:
# Replace NaN values using median

median = train['mvar1'].median()
train['mvar1'].fillna(median, inplace=True)

In [5]:
#one hot encoding of mvar47

train.mvar47[train.mvar47 == 'L'] = 0
train.mvar47[train.mvar47 == 'C'] = 1

C:\Users\SARANSH\Anaconda3\envs\saransh\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\SARANSH\Anaconda3\envs\saransh\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
test.mvar47[test.mvar47 == 'L'] = 0
test.mvar47[test.mvar47 == 'C'] = 1
test['mvar47']

C:\Users\SARANSH\Anaconda3\envs\saransh\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\SARANSH\Anaconda3\envs\saransh\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0       0
1       0
2       0
3       1
4       1
       ..
2995    0
2996    0
2997    0
2998    0
2999    0
Name: mvar47, Length: 3000, dtype: object

In [7]:
## Replace NaN values using median

for column in train:
    median = train[column].median()
    train[column].fillna(median, inplace=True)

In [8]:
# Replace NaN values using median

for column in test:
    median = test[column].median()
    test[column].fillna(median, inplace=True)

In [9]:
#converting object values to numeric to apply models

for cols in train:
    pd.to_numeric(train[cols])
    

In [10]:
train.dtypes

application_key      int64
mvar1               object
mvar2              float64
mvar3              float64
mvar4              float64
mvar5              float64
mvar6               object
mvar7               object
mvar8               object
mvar9               object
mvar10              object
mvar11              object
mvar12              object
mvar13              object
mvar14               int64
mvar15              object
mvar16              object
mvar17              object
mvar18              object
mvar19              object
mvar20              object
mvar21             float64
mvar22             float64
mvar23             float64
mvar24             float64
mvar25              object
mvar26              object
mvar27              object
mvar28              object
mvar29              object
mvar30              object
mvar31              object
mvar32              object
mvar33             float64
mvar34              object
mvar35              object
mvar36              object
m

In [11]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

Using TensorFlow backend.


In [12]:
def get_data():
    return train , test

def get_combined_data():
  #reading train data
  train , test = get_data()

  target = train.default_ind
  train.drop(['default_ind'],axis = 1 , inplace = True)

  combined = train.append(test)
  combined.reset_index(inplace=True)
  combined.drop(['application_key'], inplace=True, axis=1)
  return combined, target

#Load train and test data into pandas DataFrames
train_data, test_data = get_data()

#Combine train and test data to process them together
combined, target = get_combined_data()

In [13]:
combined.describe()


,index,mvar2,mvar3,mvar4,mvar5,mvar14,mvar21,mvar22,mvar23,mvar24,mvar33,mvar44,mvar47,mvar48
count,83000.000000,83000.000000,83000.000000,83000.000000,83000.000000,8.300000e+04,83000.000000,83000.000000,83000.000000,83000.000000,83000.000000,83000.000000,83000.000000,83000.000000
mean,38607.933735,1.016366,5.368891,0.458179,1.077025,1.894841e+05,57.756403,4158.980316,3916.838971,63.271571,6.915401,0.709832,0.627024,556.021277
std,23785.058205,1.506916,11.063304,1.699191,5.726014,7.427382e+06,30.173145,3020.723173,2719.186922,292.033418,7.552741,0.233744,0.483599,1106.953295
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.117000,0.000000,0.000000,0.000000,0.083300,0.060660,0.000000,2.000000
25%,17749.750000,0.151200,0.000000,0.000000,0.000000,4.261300e+04,39.510000,2412.605000,3189.927500,17.000000,1.250000,0.537210,0.000000,8.000000
50%,38499.500000,0.515950,0.300000,0.000000,0.000000,7.234300e+04,58.974000,3860.120000,3626.670000,39.500000,3.916700,0.738460,1.000000,17.000000
75%,59249.250000,1.303900,6.535000,0.000000,0.000000,1.189200e+05,76.117000,5306.380000,3626.670000,63.950000,10.416700,0.926713,1.000000,111.000000
max,79999.000000,31.018100,399.334000,25.754000,165.492000,9.910000e+08,1305.250000,99800.000000,124533.330000,27867.400000,76.500000,1.000000,1.000000,3247.000000


In [14]:
"""
def get_cols_with_no_nans(df,col_type):
    if (col_type == 'num'):
        predictors = df.select_dtypes(exclude=['object'])
    elif (col_type == 'no_num'):
        predictors = df.select_dtypes(include=['object'])
    elif (col_type == 'all'):
        predictors = df
    else :
        print('Error : choose a type (num, no_num, all)')
        return 0
    cols_with_no_nans = []
    for col in predictors.columns:
        if not df[col].isnull().any():
            cols_with_no_nans.append(col)
    return cols_with_no_nans
    
"""

"\ndef get_cols_with_no_nans(df,col_type):\n    if (col_type == 'num'):\n        predictors = df.select_dtypes(exclude=['object'])\n    elif (col_type == 'no_num'):\n        predictors = df.select_dtypes(include=['object'])\n    elif (col_type == 'all'):\n        predictors = df\n    else :\n        print('Error : choose a type (num, no_num, all)')\n        return 0\n    cols_with_no_nans = []\n    for col in predictors.columns:\n        if not df[col].isnull().any():\n            cols_with_no_nans.append(col)\n    return cols_with_no_nans\n    \n"

In [15]:
combined.dtypes

index       int64
mvar1      object
mvar2     float64
mvar3     float64
mvar4     float64
mvar5     float64
mvar6      object
mvar7      object
mvar8      object
mvar9      object
mvar10     object
mvar11     object
mvar12     object
mvar13     object
mvar14      int64
mvar15     object
mvar16     object
mvar17     object
mvar18     object
mvar19     object
mvar20     object
mvar21    float64
mvar22    float64
mvar23    float64
mvar24    float64
mvar25     object
mvar26     object
mvar27     object
mvar28     object
mvar29     object
mvar30     object
mvar31     object
mvar32     object
mvar33    float64
mvar34     object
mvar35     object
mvar36     object
mvar37     object
mvar38     object
mvar39     object
mvar40     object
mvar41     object
mvar42     object
mvar43     object
mvar44    float64
mvar45     object
mvar46     object
mvar47      int64
mvar48      int64
dtype: object

In [16]:
  
corr = train.corr()
corr.style.background_gradient()


,application_key,mvar2,mvar3,mvar4,mvar5,mvar14,mvar21,mvar22,mvar23,mvar24,mvar33,mvar44,mvar47,mvar48
application_key,1,0.00134955,0.0177273,-0.00333828,0.00134716,-0.000397002,0.00676143,0.0001146,-0.00432951,0.00183406,-0.00656189,-0.000587087,0.0213854,0.00434003
mvar2,0.00134955,1,0.117898,0.0908539,0.0364855,-0.00376623,0.198866,0.179913,0.170654,0.0286348,-0.128655,-0.0871349,0.102033,-0.00452186
mvar3,0.0177273,0.117898,1,0.358327,0.625564,-0.00306963,0.155329,0.109311,0.0878712,0.147608,-0.10417,-0.00903416,0.128975,-0.000312263
mvar4,-0.00333828,0.0908539,0.358327,1,0.0811323,-0.000425194,0.0965961,0.0634863,0.0515441,0.0369078,-0.0713535,0.00503268,0.0778072,-0.00496038
mvar5,0.00134716,0.0364855,0.625564,0.0811323,1,-0.00182696,0.0539883,0.0426367,0.033055,0.0148097,-0.0654411,-0.0593013,0.0581636,-0.00144024
mvar14,-0.000397002,-0.00376623,-0.00306963,-0.000425194,-0.00182696,1,-0.000884986,-0.000915541,-0.00221127,-0.00107098,-0.00229884,0.00283312,0.00392618,-0.00312026
mvar21,0.00676143,0.198866,0.155329,0.0965961,0.0539883,-0.000884986,1,0.569758,0.484041,0.103732,-0.146948,-0.151353,0.175387,0.00121603
mvar22,0.0001146,0.179913,0.109311,0.0634863,0.0426367,-0.000915541,0.569758,1,0.768689,0.0795663,-0.115791,-0.145597,0.131101,-0.00122161
mvar23,-0.00432951,0.170654,0.0878712,0.0515441,0.033055,-0.00221127,0.484041,0.768689,1,0.0654978,-0.0987494,-0.12507,0.103422,-0.00206596
mvar24,0.00183406,0.0286348,0.147608,0.0369078,0.0148097,-0.00107098,0.103732,0.0795663,0.0654978,1,-0.0157665,-0.00339132,0.0302198,-0.00620298


In [17]:
from sklearn.model_selection import train_test_split
  
#train1 = train.copy()
  
#feature_df = train1[['add_pf1',"Elevation","Aspect","Slope","Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology","Horizontal_Distance_To_Roadways","Hillshade_9am","Hillshade_Noon","Rawah","Neota"]]

feature_df = train[['mvar1','mvar2','mvar3','mvar4','mvar5','mvar6','mvar7','mvar8','mvar9','mvar10','mvar11','mvar12',"mvar13","mvar14","mvar15","mvar16","mvar17","mvar18",'mvar19','mvar20','mvar21','mvar22','mvar23','mvar24','mvar25','mvar26','mvar27','mvar28','mvar29','mvar30','mvar31','mvar32','mvar33','mvar34','mvar35','mvar36','mvar37','mvar38','mvar39','mvar40','mvar41','mvar42','mvar43','mvar44','mvar45','mvar46','mvar47','mvar48']]


x = np.asarray(feature_df)
#x = x.astype(np.int)
#y = np.asarray(train["default_ind"].astype('int'))
y = np.asarray(target) 
#x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 5)


In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense

"""
model = Sequential()

model.add(Dense(input_dim = 48, units = 200, activation='relu', kernel_initializer='uniform'))

model.add(Dense(units = 59, activation='relu', kernel_initializer='uniform'))

model.add(Dense(units = 50, activation='tanh', kernel_initializer='uniform'))
model.add(Dense(units = 90, activation='tanh', kernel_initializer='uniform'))
model.add(Dense(units = 150, activation='relu', kernel_initializer='uniform'))
model.add(Dense(units = 190, activation='relu', kernel_initializer='uniform'))
model.add(Dense(units = 250, activation='tanh', kernel_initializer='uniform'))
model.add(Dense(units = 400, activation='relu', kernel_initializer='uniform'))

model.add(Dense(units = 1, activation='sigmoid', kernel_initializer='uniform'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train,y_train,batch_size=5000, epochs=100)

model.evaluate(x_test,y_test)
"""

"\nmodel = Sequential()\n\nmodel.add(Dense(input_dim = 48, units = 200, activation='relu', kernel_initializer='uniform'))\n\nmodel.add(Dense(units = 59, activation='relu', kernel_initializer='uniform'))\n\nmodel.add(Dense(units = 50, activation='tanh', kernel_initializer='uniform'))\nmodel.add(Dense(units = 90, activation='tanh', kernel_initializer='uniform'))\nmodel.add(Dense(units = 150, activation='relu', kernel_initializer='uniform'))\nmodel.add(Dense(units = 190, activation='relu', kernel_initializer='uniform'))\nmodel.add(Dense(units = 250, activation='tanh', kernel_initializer='uniform'))\nmodel.add(Dense(units = 400, activation='relu', kernel_initializer='uniform'))\n\nmodel.add(Dense(units = 1, activation='sigmoid', kernel_initializer='uniform'))\n\nmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])\n\nmodel.fit(x_train,y_train,batch_size=5000, epochs=100)\n\nmodel.evaluate(x_test,y_test)\n"

In [25]:
params = {
    'min_child_weight': 10.0,
    'objective': 'binary:logistic',
    'max_depth': 5,
    'max_delta_step': 0.8,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'eta': 0.01,
    'gamma': 0.45,
    'eval_metric' : 'auc',
    'silent': 1,
    'num_boost_round' : 700
    }

In [26]:
from sklearn.model_selection import StratifiedKFold
kfold = 5
skf = StratifiedKFold(n_splits=kfold, random_state=42)

In [ ]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
for i, (train_index, test_index) in enumerate(skf.split(x, y)):
    print('[Fold %d/%d]' % (i + 1, kfold))
    X_train, X_valid = x[train_index], x[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    # Convert our data into XGBoost format
    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_valid, y_valid)
    d_test = xgb.DMatrix(test.values)
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]

    # Train the model! We pass in a max of 1,600 rounds (with early stopping after 70)
    # and the custom metric (maximize=True tells xgb that higher metric is better)
    mdl = xgb.train(params, d_train, 5000, watchlist, early_stopping_rounds=1000, maximize=True,verbose_eval = 1000)
    print('[Fold %d/%d Prediciton:]' % (i + 1, kfold))
    # Predict on our test data
    #p_test = mdl.predict(d_test, ntree_limit=mdl.best_ntree_limit)
    #result['loan_default'] += p_test/kfold

[Fold 1/5]
[0]	train-auc:0.688358	valid-auc:0.679005
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 1000 rounds.
[1000]	train-auc:0.817022	valid-auc:0.787848
[2000]	train-auc:0.834016	valid-auc:0.788515
Stopping. Best iteration:
[1587]	train-auc:0.827675	valid-auc:0.788571

[Fold 1/5 Prediciton:]
[Fold 2/5]
[0]	train-auc:0.690857	valid-auc:0.681015
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 1000 rounds.


In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(400, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(400, kernel_initializer='normal',activation='tanh'))
NN_model.add(Dense(400, kernel_initializer='normal',activation='sigmoid'))
NN_model.add(Dense(400, kernel_initializer='normal',activation='tanh'))
NN_model.add(Dense(400, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
NN_model.summary()

In [ ]:
NN_model.fit(train, target, epochs=50, batch_size=32, validation_split = 0.2)